# Satellite Matching with Tufts Turfs Images

Checklist 

1. [x] Set reference coordinate frame (scaling factor)
2. [x] Grab ground plane points from desired image
3. [x] Generate a 2D projection of desired ground plane points
4. [x] Grab section of satellite image as initial guess (?)
5. [ ] Downsample projection to read as an image
6. [ ] Convert image to grayscale
7. [ ] Run correlation on grayscale images
8. [ ] Run SSD on same image matchup

Other things 
1. [ ] GET MORE GRAVITY PTS

In [1]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
import imageio
%matplotlib qt
import matplotlib.pyplot as plt

from groundNAV_utils import *
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# Define Class Parameters 

images_c  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_c = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_c = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"
# Three potential images I like 
# image_i = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9587.JPEG"
# im_ID = 145
image_i = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG"
im_ID = 4
# image_i = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9557.JPEG"
# im_ID = 115
# image_i = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9570.JPEG" # Better than 9557
# im_ID = 128

# Load in satellite reference image
sat_ref_o = cv2.imread('TTurf/TurfSat.jpg')

In [3]:
# Create class d
gnav = gNAV_agent(images_c, cameras_c, pts3d_c, image_i)

# Grab raw points and RGB data 
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d)

In [4]:
def find_image_id_by_path(images, image_path):
    target_name = os.path.basename(image_path)  # Extract just the filename
    for image_id, image in images.items():
        if image.name == target_name:  # Match the exact filename
            return image_id
    return None  # Return None if not found

# Example usage:
image_i = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG"
image_id = find_image_id_by_path(gnav.images, image_i)
print(image_id)

4


In [5]:
# # View reference satellite image
# # Make figure 
# fig, ax = plt.subplots(figsize=(15,8))

# # Change color scheme to match matplotlib 
# sat_ref_plt = cv2.cvtColor(sat_ref_o, cv2.COLOR_BGR2RGB)

# # Plot 
# ax.imshow(sat_ref_plt)
# ax.axis("off")

# plt.show()

### Set reference coordinate frame 

In [6]:
# Define origin 
origin_w = np.array([0,0,0])
# Find GRAVITY and HEIGHT
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
# pts_gnd_idx = np.array([7389, 7387, 7385, 7379, 7375])
pts_gnd_T = scene_pts[pts_gnd_idx]

grav_dir = gnav.grav_SVD(pts_gnd_idx)
# REVERSE DIRECTION IF NECESSARY 
# grav_dir = -grav_dir
# gnav.grav_vec = grav_dir
print('Gravity vector \n', grav_dir)

h_0 = gnav.height_avg(pts_gnd_idx, origin_w)
print('\nHeight h_0 = ', h_0)

Gravity vector 
 [ 0.98974383 -0.02606363  0.1404558 ]

Height h_0 =  0.2729831012742149


In [7]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Ground plane
# GND = o3d.geometry.PointCloud()
# GND.points = o3d.utility.Vector3dVector(pts_gnd_T)
# GND.paint_uniform_color([1,0,0])

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# # vis.add_geometry(scene_cloud)
# vis.add_geometry(GND)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [8]:
focal = gnav.cameras[2].params[0]
gnav.focal = focal
print(focal)

2987.396087478296


In [9]:
# Define coordinate frame
z_bar = grav_dir
# Define a vector on the x-y plane 
P1, P2 = scene_pts[pts_gnd_idx[0],:], scene_pts[pts_gnd_idx[5],:]
v = P2-P1
# X direction as ZxV
x_dir = np.cross(z_bar, v)
x_bar = x_dir/np.linalg.norm(x_dir) # Normalize
print("X unit vector \n", x_bar)
# Y direction as ZxX
y_dir = np.cross(z_bar, x_bar)
y_bar = y_dir/np.linalg.norm(y_dir) # Normalize
print("\nY unit vector \n", y_bar)

# Rotation matrix 
rotmat = np.column_stack((x_bar, y_bar, z_bar))
print("\nRotation matrix \n", rotmat)
# Translation vector 
trans = P1.reshape([3,1])

# Form transformation matrix
bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
tform = np.concatenate([np.concatenate([rotmat, trans], 1), bottom], 0)
print("\nTransformation matrix to ground \n", tform)

# Translation from ground to desired height
h = 1
x = 0
y = 0
yaw = np.deg2rad(220)
trans2 = np.array([x,y,-h]).reshape([3,1])
# Rotation matrix
euler_angles = [0., 0., yaw] #rotation about xaxis, yaxis, zaxis
rotmat2 = R.from_euler('xyz', euler_angles).as_matrix()
# rotmat2 = np.eye(3)
tform2 = np.concatenate([np.concatenate([rotmat2, trans2], 1), bottom], 0)
print("\nTransformation matrix from ground desired coord frame \n", tform2)

# Combine transformation matrices 
tform_T = tform @ tform2
print("\nTransformation from origin to desired coord frame \n", tform_T)

X unit vector 
 [-0.00155069  0.98119701  0.19300266]

Y unit vector 
 [-0.14284517 -0.191241    0.97109327]

Rotation matrix 
 [[-0.00155069 -0.14284517  0.98974383]
 [ 0.98119701 -0.191241   -0.02606363]
 [ 0.19300266  0.97109327  0.1404558 ]]

Transformation matrix to ground 
 [[-1.55069060e-03 -1.42845166e-01  9.89743833e-01 -3.22342664e-01]
 [ 9.81197008e-01 -1.91240997e-01 -2.60636319e-02 -6.79877281e-01]
 [ 1.93002661e-01  9.71093270e-01  1.40455805e-01  4.08087397e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Transformation matrix from ground desired coord frame 
 [[-0.76604444  0.64278761  0.          0.        ]
 [-0.64278761 -0.76604444  0.          0.        ]
 [ 0.          0.          1.         -1.        ]
 [ 0.          0.          0.          1.        ]]

Transformation from origin to desired coord frame 
 [[ 0.093007    0.10842898  0.98974383 -1.3120865 ]
 [-0.62871317  0.77720038 -0.02606363 -0.65381365]
 [-0.77205534 -0.61984088  0.1404

In [10]:
# Transform all point to the new coordinate system 
tform_T_inv = gnav.inv_homog_transform(tform_T)
__, pts_new, pts_vec = gnav.unit_vec_tform(scene_pts, origin_w, tform_T_inv)
print(min(pts_new[:,2]))

-28.06557783058717


In [11]:
origin_w = np.array([0,0,0,1]).reshape(-1,1)
# print(origin_w)
origin_ref = tform_T_inv @ origin_w
origin_ref = origin_ref.reshape([4])
origin_ref = origin_ref[:-1]
print(origin_ref)

[2.75319285 3.0928447  0.72813416]


In [12]:
# Create 2D projection 
pts_2D = np.zeros((len(pts_new),3))
for i in range(len(pts_new)):
    pt = pts_new[i,:]
    z = pts_new[i,2]
    pt = pt/z
    pts_2D[i,:] = pt

print(pts_2D)

[[-12.75735292  20.13222338   1.        ]
 [-29.55518524  44.66394118   1.        ]
 [-38.81332534  40.63886308   1.        ]
 ...
 [  0.57933457  -1.85324267   1.        ]
 [  0.08889363  -2.64329522   1.        ]
 [ -0.34118845  -3.33028598   1.        ]]


In [13]:
# Crop points outside of boundary 
# print(max(pts_2D[:,1]))
x_min = -1
x_max = 1
y_min = -1
y_max = 1

boundary_pts_idx = np.argwhere((pts_2D[:,0] < x_min) | (pts_2D[:,0] > x_max) | (pts_2D[:,1] < y_min) | (pts_2D[:,1] > y_max))
new_pts_idx = np.argwhere((pts_2D[:,0] > x_min) & (pts_2D[:,0] < x_max) & (pts_2D[:,1] > y_min) & (pts_2D[:,1] < y_max))
print("\nIDX outside boundary \n", boundary_pts_idx)
boundary_pts = pts_2D[boundary_pts_idx[:,0],:]
new_pts = pts_2D[new_pts_idx[:,0],:]
print("\nPts outside boundary \n", boundary_pts)


IDX outside boundary 
 [[     0]
 [     1]
 [     2]
 ...
 [272113]
 [272114]
 [272115]]

Pts outside boundary 
 [[-12.75735292  20.13222338   1.        ]
 [-29.55518524  44.66394118   1.        ]
 [-38.81332534  40.63886308   1.        ]
 ...
 [  0.57933457  -1.85324267   1.        ]
 [  0.08889363  -2.64329522   1.        ]
 [ -0.34118845  -3.33028598   1.        ]]


In [14]:
pts_2D = new_pts
rgb_data_2D = rgb_data[new_pts_idx[:,0],:]

### Grab gound plane points from desired image

In [15]:
# Define boundaries for selected ground plane points 
# x,y = 150,1450 # Top left corner of square
x,y = 150, 1600
side_x = 2500 # WIDTH
side_y = 1000 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y) 

# Get necessary location and rghb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y)
# print(pts_loc)

### Generate 2D Projection of Desired ground Plane points 

We want:
- Ground plane pts (currently in camera coords)
- Ground plane points in world coords (full process)
- Transformed to reference frame (pts*tform_T)
- Get projection in 2D 

In [16]:
# Unit vectors in world coords
pts_vec_c, pts_rgb_gnd = gnav.unit_vec_c(pts_loc, pts_rgb)

# Get transformation matrix that move from camera coords to world coords 
homog_c2w = gnav.get_pose_id(im_ID)
print('Homogeneous transformation from world to camera \n', homog_c2w)

# Inverse for world coords to camera coords
homog_w2c = gnav.inv_homog_transform(homog_c2w)
print('\n Homogeneous transformation from camera to world \n', homog_w2c)

# Transform to world coords 
origin_w = np.array([0,0,0])
origin_c, pts_loc_w, pts_vec_w = gnav.unit_vec_tform(pts_vec_c, origin_w, homog_c2w)
print('\n New camera frame origin = ', origin_c)

# Get new points 
ranges, new_pts = gnav.pt_range(pts_vec_w, homog_c2w, origin_c)
print('\nNew Points \n', new_pts)

Homogeneous transformation from world to camera 
 [[ 9.98279206e-01 -5.03723739e-02  3.00208235e-02  8.07279503e-01]
 [ 5.86330766e-02  8.65188026e-01 -4.98007873e-01  7.97854170e-01]
 [-8.87818184e-04  4.98911118e-01  8.66652703e-01 -5.22152171e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 Homogeneous transformation from camera to world 
 [[ 9.98279206e-01  5.86330766e-02 -8.87818184e-04 -8.57306748e-01]
 [-5.03723739e-02  8.65188026e-01  4.98911118e-01  1.95544595e+00]
 [ 3.00208235e-02 -4.98007873e-01  8.66652703e-01  4.89834837e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 New camera frame origin =  [ 0.8072795   0.79785417 -5.22152171]

New Points 
 [[-0.35073014 -0.18256438  4.38114898]
 [-0.35054743 -0.18507643  4.37939531]
 [-0.35036472 -0.18758833  4.37764174]
 ...
 [ 0.96286393  0.27795001 -4.78984224]
 [ 0.96287434  0.27776646 -4.78994968]
 [ 0.96288475  0.2775829  -4.79005713]]


In [17]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Create point cloud for scene pts
# cam_cloud = o3d.geometry.PointCloud()
# cam_cloud.points = o3d.utility.Vector3dVector(new_pts)
# cam_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)

# # Add necessary geometries to visualization 
# # vis.add_geometry(axis_origin)
# # vis.add_geometry(scene_cloud)
# vis.add_geometry(cam_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [18]:
# Transform points to reference coords
# Transform all point to the new coordinate system 
tform_T_inv = gnav.inv_homog_transform(tform_T)
__, loc_128_pts_o, loc_128_vec = gnav.unit_vec_tform(new_pts, origin_w, tform_T_inv)
# print(min(pts_new[:,2]))

### Grab section of Satellite image for initial guess

In [23]:
# Create 2D points from image (1 in z direction)
# print(sat_im_crop)
sat_ref = cv2.cvtColor(sat_ref_o, cv2.COLOR_BGR2GRAY)
L = sat_ref.shape[0]
W = sat_ref.shape[1]
n = L*W
# print(L,W)
sat_pts = np.zeros((n,3))
sat_rgb = np.zeros((n,3))
# print(sat_pts)
count = 0
for i in range(L):
    for j in range(W):
        px = j
        py = j
        rgb = sat_ref[i][j]
        # print(rgb)
        sat_pts[count] = [j, i, 1]
        sat_rgb[count] = rgb
        count += 1

focal = 2987.396087478296
print("\n Focal length = ", focal)
sat_rgb = sat_rgb/255
print("Satellite points\n", sat_pts)
print("\nRGB values \n", sat_rgb)


 Focal length =  2987.396087478296
Satellite points
 [[0.000e+00 0.000e+00 1.000e+00]
 [1.000e+00 0.000e+00 1.000e+00]
 [2.000e+00 0.000e+00 1.000e+00]
 ...
 [1.554e+03 1.195e+03 1.000e+00]
 [1.555e+03 1.195e+03 1.000e+00]
 [1.556e+03 1.195e+03 1.000e+00]]

RGB values 
 [[0.50588235 0.50588235 0.50588235]
 [0.52941176 0.52941176 0.52941176]
 [0.50980392 0.50980392 0.50980392]
 ...
 [0.80392157 0.80392157 0.80392157]
 [0.93333333 0.93333333 0.93333333]
 [0.95686275 0.95686275 0.95686275]]


In [24]:
print(loc_128_pts_o)

[[-0.54713634  0.19731102  1.00111726]
 [-0.54418605  0.19646547  1.00111726]
 [-0.54123594  0.19561996  1.00111726]
 ...
 [ 6.36601838  6.38220996  1.00111726]
 [ 6.3662177   6.38213502  1.00111726]
 [ 6.36641702  6.38206009  1.00111726]]


In [25]:
# Create a function which makes a transformation matrix
def tform_create(x,y,z,roll,pitch,yaw):
    """ 
    Creates a transformation matrix 
    Inputs: translation in x,y,z, rotation in roll, pitch, yaw (DEGREES)
    Output: Transformation matrix (4x4)
    """
    # Rotation
    roll_r, pitch_r, yaw_r = np.deg2rad([roll, pitch, yaw])
    euler_angles = [roll_r, pitch_r, yaw_r]
    rotmat = R.from_euler('xyz', euler_angles).as_matrix()

    # Translation
    trans = np.array([x,y,z]).reshape([3,1])

    # Create 4x4
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1,4])
    tform = np.concatenate([np.concatenate([rotmat, trans], 1), bottom], 0)
    # print("\nTransformation matrix \n", tform)
    
    return tform 

In [26]:
# Implementing an initial guess for the local image 

# SCALE
loc_128_pts = loc_128_pts_o.copy()
# loc_128_pts[:, :2] *= focal/100
loc_128_pts[:, :2] *= focal/46

# Translation and Rotation for initial guess
x = 400
# x = 0
y = 295
# y = 0
yaw = 0
# yaw = 0

tform = tform_create(x,y,0,0,0,yaw)

# Get new points 
__, loc_128_pts_tf, loc_128_vec = gnav.unit_vec_tform(loc_128_pts, origin_w, tform)
# print(min(pts_new[:,2]))

In [27]:
# Convert RGB to intensity (grayscale)
intensity = 0.299 * pts_rgb_gnd[:, 0] + 0.587 * pts_rgb_gnd[:, 1] + 0.114 * pts_rgb_gnd[:, 2]
# print(intensity)
# Create grayscale color representation
gray_colors = np.tile(intensity[:, np.newaxis], (1, 3))  # Repeat intensity across R, G, B channels
print(gray_colors)


[[0.3096     0.3096     0.3096    ]
 [0.30175686 0.30175686 0.30175686]
 [0.29391373 0.29391373 0.29391373]
 ...
 [0.47094902 0.47094902 0.47094902]
 [0.42781176 0.42781176 0.42781176]
 [0.39643922 0.39643922 0.39643922]]


In [29]:
# PLOT SATELLITE AND RGB IMAGE - INIT GUESS - OPEN3d

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# Create point cloud for satellite reference 
satref_cloud = o3d.geometry.PointCloud()
satref_cloud.points = o3d.utility.Vector3dVector(sat_pts)
satref_cloud.colors = o3d.utility.Vector3dVector(sat_rgb)

# Create point cloud for local image
loc_cloud = o3d.geometry.PointCloud()
loc_cloud.points = o3d.utility.Vector3dVector(loc_128_pts_tf)
# loc_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)
loc_cloud.colors = o3d.utility.Vector3dVector(gray_colors)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(loc_cloud)
vis.add_geometry(satref_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 81
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 81
pci id for fd 81: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# Assign wedge points to a nearest neighbor pixel value

In [ ]:
# Process 
# NOTE: 2.5 million points - choose every 10th
# Iterate through each point for a location
# VERIFY DIFFS TO CONFIRM

In [ ]:
# Build KD tree for the sat_pts for nearest neighbor approach 
tree = cKDTree(sat_pts)

diffs = np.zeros((int(len(loc_128_pts_tf)/100), 1))
print(diffs.shape)

# Nearest neighbor 
count = 0
for i in range(0, len(loc_128_pts_tf), 100):
# for i in range(5): # Sanity check 
    locx, locy = loc_128_pts_tf[i,:-1]
    intensity = gray_colors[i,0]
    # print("Intensity", intensity)
    # Find nearest neighbor in sat_pts
    dist, idx = tree.query([locx, locy, 1])
    nearest_x, nearest_y = sat_pts[idx,:-1]
    intensity_ref = sat_rgb[idx,0]
    # print("Reference intensity", intensity_ref)
    dif = intensity_ref - intensity
    # print("Difference", dif)
    diffs[count] = dif
    # print("\nLocation x and y = ", locx, locy)
    # print("\nReference location = ", nearest_x, nearest_y)
    # print("\nIndex of point = ", idx)
    # print("\nPrinting the point: ", sat_pts[idx,:])
    count += 1
    
print(diffs)
ssd = np.sum(diffs**2)
print("\n SSD \n", ssd)

In [ ]:
# Loop through a grid of shifts 
# nxn shift grid
n = 5
# Copy the inital guess (tf point)
loc_128_pts_inig = loc_128_pts_tf.copy()

# Location tree of satellite points 
tree = cKDTree(sat_pts)

# Keep track of SSDs
ssds = np.zeros((2*n+1,2*n+1))
print(ssds)

for shiftx in range(-n,n+1):
    for shifty in range(-n,n+1):
        # shiftx = i
        # shifty = j
        print(shiftx, shifty)
        # Difference values
        diffs = np.zeros((int(len(loc_128_pts_inig)/100), 1))
        # Transformation matrix for shift
        tform = tform_create(shiftx, shifty, 0, 0, 0, 0)
        # Transform points
        __, loc_128_pts_curr, loc_128_vec = gnav.unit_vec_tform(loc_128_pts_inig, origin_w, tform)
        # print(shiftx, shifty, loc_128_pts_curr)
        # SSD process for each transformation
        count = 0
        for i in range(0, len(loc_128_pts_curr),100):
            # Location of current point
            locx, locy = loc_128_pts_curr[i,:-1]
            intensity = gray_colors[i,0]
            # print(locx,locy)
            # Distance and index of nearest neighbor
            dist, idx = tree.query([locx, locy, 1])
            nearest_x, nearest_y = sat_pts[idx,:-1]
            intensity_ref = sat_rgb[idx,0]
            # print("Reference intensity", intensity_ref)
            dif = intensity_ref - intensity
            diffs[count] = dif
            count += 1
        # Sum of squared differences
        ssd_curr = np.sum(diffs**2)
        print("\n Current SSD \n", ssd_curr)
        ssds[shiftx+n, shifty+n] = ssd_curr

print(f"\nSSDs for {n}x{n} pixel shifts\n", ssds)

In [ ]:
# print(loc_128_pts_curr)
print(loc_128_pts_inig)

In [ ]:
# Plot and visualize SSD values 

# Create a 5x5 grid of x and y coordinates
x = np.linspace(-n, n, 2*n+1)
y = np.linspace(-n, n, 2*n+1)
X, Y = np.meshgrid(x, y)
# print(ssds)

# print(x)
# print(X)
# print(Y)



In [ ]:
# Create the figure and 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D vectors
ax.quiver(X, Y, np.zeros_like(ssds), np.zeros_like(ssds), np.zeros_like(ssds), (ssds/2000)**2)

# Set axis limits
ax.set_xlim([-8, 8])  # X axis range
ax.set_ylim([-8, 8])  # Y axis range
ax.set_zlim([0, (np.max(ssds) / 2000)**2])  # Z axis range, adjust based on your data

# Labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Pixel Correction Vector Field')

In [ ]:
# PLOT SATELLITE AND RGB IMAGE - INIT GUESS - OPEN3d

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# Create point cloud for satellite reference 
satref_cloud = o3d.geometry.PointCloud()
satref_cloud.points = o3d.utility.Vector3dVector(sat_pts)
satref_cloud.colors = o3d.utility.Vector3dVector(sat_rgb)

# Create point cloud for local image
loc_cloud = o3d.geometry.PointCloud()
loc_cloud.points = o3d.utility.Vector3dVector(loc_128_pts_inig)
# loc_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)
loc_cloud.colors = o3d.utility.Vector3dVector(gray_colors)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(loc_cloud)
vis.add_geometry(satref_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

# Plotting Tools 

In [ ]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(pts_new)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# Plot all points on new coord system 
# Plot gravity vector in open3d

# Use open3d to create point cloud visualization 

# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# Create point cloud for scene pts
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(pts_new)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
vis.add_geometry(scene_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

In [ ]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="2D scene with cropped out points in RED")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(pts_2D)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data_2D)

# # Create point cloud for points outside boundary 
# scene_boundary = o3d.geometry.PointCloud()
# scene_boundary.points = o3d.utility.Vector3dVector(boundary_pts)
# scene_boundary.paint_uniform_color([1,0,0]) # RGB

# # Add necessary geometries to visualization 
# # vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)
# vis.add_geometry(scene_boundary)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="2D projection (removed outliers)")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(pts_2D)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data_2D)

# # Create point cloud for points outside boundary 
# scene_boundary = o3d.geometry.PointCloud()
# scene_boundary.points = o3d.utility.Vector3dVector(boundary_pts)
# scene_boundary.paint_uniform_color([1,0,0]) # RGB

# # Add necessary geometries to visualization 
# # vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)
# # vis.add_geometry(scene_boundary)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Create point cloud for scene pts
# cam_cloud = o3d.geometry.PointCloud()
# cam_cloud.points = o3d.utility.Vector3dVector(new_pts)
# cam_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)
# vis.add_geometry(cam_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # Plot all points on new coord system 
# # Plot gravity vector in open3d

# # Use open3d to create point cloud visualization 

# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene pts
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(pts_new)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Create point cloud for scene pts
# cam_cloud = o3d.geometry.PointCloud()
# cam_cloud.points = o3d.utility.Vector3dVector(loc_128_pts)
# cam_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# # vis.add_geometry(scene_cloud)
# vis.add_geometry(cam_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOT NEW POINTS 

# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=500)

# # Create point cloud for satellite reference 
# satref_cloud = o3d.geometry.PointCloud()
# satref_cloud.points = o3d.utility.Vector3dVector(sat_pts)
# satref_cloud.colors = o3d.utility.Vector3dVector(sat_rgb)

# # Create point cloud for local image
# loc_cloud = o3d.geometry.PointCloud()
# loc_cloud.points = o3d.utility.Vector3dVector(loc_128_pts_tf)
# loc_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(loc_cloud)
# vis.add_geometry(satref_cloud)

# render_option = vis.get_render_option()
# render_option.point_size = 2

# vis.poll_events()
# vis.update_renderer()

# # Set up initial viewpoint
# view_control = vis.get_view_control()
# # Direction which the camera is looking
# view_control.set_front([0, 0, -1])  # Set the camera facing direction
# # Point which the camera revolves about 
# view_control.set_lookat([0, 0, 0])   # Set the focus point
# # Defines which way is up in the camera perspective 
# view_control.set_up([1, 0, 0])       # Set the up direction
# view_control.set_zoom(1)           # Adjust zoom if necessary


# # Capture frames for GIF
# frames = []
# num_frames = 30  # Adjust the number of frames
# angle_step = 180/num_frames


# for i in range(num_frames):
# 	# Rotate the view
#     view_control.rotate(angle_step, 0)  # (horizontal, vertical)

#     # vis.update_geometry(axis_orig) # Only if I move it myself?
#     vis.poll_events()
#     vis.update_renderer()

#     # Capture frame directly into memory
#     image = vis.capture_screen_float_buffer(False)
#     image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
#     frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)
    
# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)



# # Create GIF
# # Ensure frames are in the correct format
# frames = [frame.astype("uint8") for frame in frames]

# # Use imageio to save as GIF
# imageio.mimsave("Initial_placement.gif", frames, fps=30, loop=0)  # Adjust fps if necessary

# # Run visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOT SATELLITE AND RGB IMAGE - INIT GUESS - OPEN3d

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# # Create point cloud for satellite reference 
# satref_cloud = o3d.geometry.PointCloud()
# satref_cloud.points = o3d.utility.Vector3dVector(sat_pts)
# satref_cloud.colors = o3d.utility.Vector3dVector(sat_rgb)

# # Create point cloud for local image
# loc_cloud = o3d.geometry.PointCloud()
# loc_cloud.points = o3d.utility.Vector3dVector(loc_128_pts_curr)
# # loc_cloud.colors = o3d.utility.Vector3dVector(pts_rgb_gnd)
# loc_cloud.colors = o3d.utility.Vector3dVector(gray_colors)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(loc_cloud)
# vis.add_geometry(satref_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()